In [3]:
import numpy as np
import plotly.graph_objects as go
import notebooks.latex_utils as latex

# P = np.array([ [x, np.sqrt(x)] for x in np.linspace(0, 100, 3)])
P = np.array([
    [0, 0],
    [1, 1],
    [3, -6],
    [4, -2.5],
    [5, 3],
    [6, -0.8],
    [7, 2],
    [9, 1.5],
    [10, -0.5]
])


# tau = 10
def interpolate(P, tau):
    n = P.shape[0] - 1

    X = P[:, 0]
    Y = P[:, 1]
    print(f"X = {X.shape} {X}")
    print(f"Y = {Y}")
    h = np.zeros(n)
    for i in range(n):
        h[i] = X[i + 1] - X[i]

    print(f"h = {h}")

    alpha = np.zeros(n)
    for i in range(n):
        alpha[i] = 1 / h[i] - tau / np.sinh(tau * h[i])
    print(f"alpha = {alpha}")

    beta = np.zeros(n)
    for i in range(n):
        beta[i] = tau * (1.0 / np.tanh(tau * h[i])) - 1 / h[i]
    print(f"beta = {beta}")

    gamma = np.zeros(n)
    for i in range(n):
        gamma[i] = tau ** 2 / h[i] * (Y[i + 1] - Y[i])
    print(f"gamma = {gamma}")

    C = np.zeros(n - 1)
    for i in range(n-1):
        C[i] = gamma[i+1] - gamma[i]
    A = np.zeros((n - 1, n-1))

    for i in range(n-1):
        if i == 0:
            A[i, i] = beta[i] + beta[i+1]
        elif i == n-2:
            A[i, i-1] = alpha[n-2]
            A[i, i] = beta[n-2] + beta[n-1]
        else:
            A[i, i-1] = alpha[i]
            A[i, i] = beta[i] + beta[i+1]
            A[i, i+1] = alpha[i+1]

    z = np.concatenate(([0], np.linalg.solve(A, C), [0]))
    print(f"z = {z}")

    tau2 = tau ** 2
    def s(x):
        i = 0
        for j in range(n, -1, -1):
            if x - X[j] >= 0:
                i = j
                break
        if i >= n:
            i = n - 1
        return (z[i] * np.sinh(tau * (X[i + 1] - x)) + z[i + 1] * np.sinh(tau * (x - X[i]))) / (tau2 * np.sinh(tau * h[i])) + \
            (Y[i] - z[i]/tau2) * (X[i+1] - x) / h[i] + (Y[i+1] - z[i+1]/tau2)* (x - X[i])/h[i]
    return s

S1 = interpolate(P, 0.01)
S2 = interpolate(P, 10)

n = P.shape[0] - 1
X = P[:, 0]
Y = P[:, 1]
fig1 = go.Scatter(x=X, y=Y, name='Data points')
fig2 = go.Scatter(x=X, y=np.zeros(P.shape[0]), mode='markers', line=dict(color='red'))
XX = np.linspace(X[0], X[n], 1000)
YY1 = [S1(x) for x in XX]
fig3 = go.Scatter(x=XX, y=YY1, line=dict(color='white'), name='Interpolation')
YY2 = [S2(x) for x in XX]
fig4 = go.Scatter(x=XX, y=YY2, line=dict(color='green'), name='Interpolation')

fig = go.Figure(fig1)
fig.add_trace(fig2)
fig.add_trace(fig3)
fig.add_trace(fig4)
fig.show()


X = (9,) [ 0.  1.  3.  4.  5.  6.  7.  9. 10.]
Y = [ 0.   1.  -6.  -2.5  3.  -0.8  2.   1.5 -0.5]
h = [1. 2. 1. 1. 1. 1. 2. 1.]
alpha = [1.66664722e-05 3.33317778e-05 1.66664722e-05 1.66664722e-05
 1.66664722e-05 1.66664722e-05 3.33317778e-05 1.66664722e-05]
beta = [3.33331111e-05 6.66648890e-05 3.33331111e-05 3.33331111e-05
 3.33331111e-05 3.33331111e-05 6.66648890e-05 3.33331111e-05]
gamma = [ 1.0e-04 -3.5e-04  3.5e-04  5.5e-04 -3.8e-04  2.8e-04 -2.5e-05 -2.0e-04]


$$\begin{align}\begin{bmatrix}9.99980000699896e-05&0.0&0.0&0.0&0.0&0.0&0.0\\3.33317778433706e-05&9.99980000699896e-05&1.6666472224313722e-05&0.0&0.0&0.0&0.0\\0.0&1.6666472224313722e-05&6.6666222226619e-05&1.6666472224313722e-05&0.0&0.0&0.0\\0.0&0.0&1.6666472224313722e-05&6.6666222226619e-05&1.6666472224313722e-05&0.0&0.0\\0.0&0.0&0.0&1.6666472224313722e-05&6.6666222226619e-05&1.6666472224313722e-05&0.0\\0.0&0.0&0.0&0.0&1.6666472224313722e-05&9.99980000699896e-05&3.33317778433706e-05\\0.0&0.0&0.0&0.0&0.0&3.33317778433706e-05&9.99980000699896e-05\\\end{bmatrix}\cdot z = \begin{bmatrix}-0.00045&0.0007&0.00020000000000000004&-0.00093&0.00066&-0.000305&-0.000175\\\end{bmatrix}\end{align}$$

z = [  0.          -4.50009      7.4997535    6.00220895 -19.50856933
  16.23180755  -5.81852349   0.18942111   0.        ]
X = (9,) [ 0.  1.  3.  4.  5.  6.  7.  9. 10.]
Y = [ 0.   1.  -6.  -2.5  3.  -0.8  2.   1.5 -0.5]
h = [1. 2. 1. 1. 1. 1. 2. 1.]
alpha = [0.999092   0.49999996 0.999092   0.999092   0.999092   0.999092
 0.49999996 0.999092  ]
beta = [9.00000004 9.5        9.00000004 9.00000004 9.00000004 9.00000004
 9.5        9.00000004]
gamma = [ 100. -350.  350.  550. -380.  280.  -25. -200.]


$$\begin{align}\begin{bmatrix}18.500000041223075&0.0&0.0&0.0&0.0&0.0&0.0\\0.49999995877692754&18.500000041223075&0.9990920014028788&0.0&0.0&0.0&0.0\\0.0&0.9990920014028788&18.00000008244615&0.9990920014028788&0.0&0.0&0.0\\0.0&0.0&0.9990920014028788&18.00000008244615&0.9990920014028788&0.0&0.0\\0.0&0.0&0.0&0.9990920014028788&18.00000008244615&0.9990920014028788&0.0\\0.0&0.0&0.0&0.0&0.9990920014028788&18.500000041223075&0.49999995877692754\\0.0&0.0&0.0&0.0&0.0&0.49999995877692754&18.500000041223075\\\end{bmatrix}\cdot z = \begin{bmatrix}-450.0&700.0&200.0&-930.0&660.0&-305.0&-175.0\\\end{bmatrix}\end{align}$$

z = [  0.         -24.32432427  37.84498698  12.04083351 -54.59520026
  40.72068305 -18.44341697  -8.96098875   0.        ]
